In [ ]:
from modeling import armar_dataset_modelado
from src.utils import preprocesamiento_completo
# Para entrenamiento (modo completo)
preprocesamiento_completo()
armar_dataset_modelado(test=False)
# preprocesamiento_completo(test=True, path_test="pathtest")
# armar_dataset_modelado(test=True, path_test="pathtest")

🧱 Preprocesamiento en modo TRAIN (años 2020–2024)
